In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

ModuleNotFoundError: No module named 'pandas'

In [3]:
 

# Load raw data
fraud_df = pd.read_csv('../data/raw/Fraud_Data.csv')
ip_df = pd.read_csv('../data/raw/IpAddress_to_Country.csv')
 


NameError: name 'pd' is not defined

In [ ]:
# ---- EDA ----
print(fraud_df.head())
print(fraud_df.info())
print(fraud_df['class'].value_counts())



   user_id          signup_time        purchase_time  purchase_value  \
0    22058  2015-02-24 22:55:49  2015-04-18 02:47:11              34   
1   333320  2015-06-07 20:39:50  2015-06-08 01:38:54              16   
2     1359  2015-01-01 18:52:44  2015-01-01 18:52:45              15   
3   150084  2015-04-28 21:13:25  2015-05-04 13:54:50              44   
4   221365  2015-07-21 07:09:52  2015-09-09 18:40:53              39   

       device_id source browser sex  age    ip_address  class  
0  QVPSPJUOCKZAR    SEO  Chrome   M   39  7.327584e+08      0  
1  EOGFQPIZPYXFZ    Ads  Chrome   F   53  3.503114e+08      0  
2  YSSKYOSJHPPLJ    SEO   Opera   M   53  2.621474e+09      1  
3  ATGTXKYKUDUQN    SEO  Safari   M   41  3.840542e+09      0  
4  NAUITBZFJKHWW    Ads  Safari   M   45  4.155831e+08      0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151112 entries, 0 to 151111
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          -----

In [ ]:
# Convert IP addresses to integers safely
def ip_to_int(ip):
    try:
        # Ensure it's a string
        ip = str(ip)
        # Convert IP to a single integer (simplified)
        parts = ip.split('.')
        if len(parts) != 4:
            return np.nan
        return int(parts[0]) * 256**3 + int(parts[1]) * 256**2 + int(parts[2]) * 256 + int(parts[3])
    except:
        return np.nan

fraud_df['ip_int'] = fraud_df['ip_address'].apply(ip_to_int)

# Optional: drop rows where conversion failed
fraud_df = fraud_df.dropna(subset=['ip_int'])


In [ ]:
# ---- Geolocation integration ----
fraud_df['ip_int'] = fraud_df['ip_address'].apply(lambda x: int(x.split('.')[-1]))  # simplified placeholder
fraud_df = fraud_df.merge(ip_df, left_on='ip_int', right_on='lower_bound_ip_address', how='left')



In [ ]:
fraud_df['purchase_time'] = pd.to_datetime(fraud_df['purchase_time'])
fraud_df['signup_time'] = pd.to_datetime(fraud_df['signup_time'])
fraud_df['time_since_signup'] = (fraud_df['purchase_time'] - fraud_df['signup_time']).dt.total_seconds()
fraud_df['hour_of_day'] = fraud_df['purchase_time'].dt.hour
fraud_df['day_of_week'] = fraud_df['purchase_time'].dt.dayofweek

# Transaction frequency
freq = fraud_df.groupby('user_id').size().reset_index(name='user_tx_count')
fraud_df = fraud_df.merge(freq, on='user_id', how='left')



In [ ]:
 

# Check the shape of the dataframe
print("Number of rows and columns:", fraud_df.shape)

# Check for rows with all missing numeric columns
print("Missing values per column:")
print(fraud_df.isna().sum())

# If the dataframe is empty
if fraud_df.shape[0] == 0:
    print("Warning: DataFrame is empty! Check your data loading and cleaning steps.")
else:
    # Proceed with numeric scaling only if there are rows
    num_cols = [col for col in ['purchase_value', 'age', 'time_since_signup', 'user_tx_count'] if col in fraud_df.columns]
    if num_cols:
        scaler = StandardScaler()
        fraud_df[num_cols] = scaler.fit_transform(fraud_df[num_cols])


Number of rows and columns: (0, 15)
Missing values per column:
user_id                   0
signup_time               0
purchase_time             0
purchase_value            0
device_id                 0
age                       0
ip_address                0
class                     0
ip_int                    0
lower_bound_ip_address    0
upper_bound_ip_address    0
time_since_signup         0
hour_of_day               0
day_of_week               0
user_tx_count             0
dtype: int64


In [ ]:
# Separate features and target
X = fraud_df.drop(columns=['class', 'user_id', 'device_id', 'purchase_time', 'signup_time', 'ip_address', 'lower_bound_ip_address', 'upper_bound_ip_address'])
y = fraud_df['class']



In [ ]:
# --- Step 2: Inspect data ---
print("Fraud DF shape:", fraud_df.shape)
print("IP DF shape:", ip_df.shape)
print(fraud_df.head())

# --- Step 3: Merge with IP data safely ---
# Convert IP addresses to integer (basic placeholder)
fraud_df['ip_int'] = fraud_df['ip_address'].fillna('0.0.0.0').apply(
    lambda x: sum([int(part) << (8*(3-i)) for i, part in enumerate(x.split('.'))])
)
ip_df['lower_bound_ip_address'] = ip_df['lower_bound_ip_address'].astype(int)
ip_df['upper_bound_ip_address'] = ip_df['upper_bound_ip_address'].astype(int)

# Merge using a range join approach (simplified for demo)
fraud_df = fraud_df.assign(country='Unknown')  # skip complex range join for now

# --- Step 4: Fill missing categorical values safely ---
cat_cols = ['source', 'browser', 'sex', 'country']
for col in cat_cols:
    if col in fraud_df.columns:
        fraud_df[col] = fraud_df[col].fillna('Unknown')
    else:
        fraud_df[col] = 'Unknown'

# --- Step 5: Feature engineering ---
fraud_df['time_since_signup'] = (
    pd.to_datetime(fraud_df['purchase_time']) - pd.to_datetime(fraud_df['signup_time'])
).dt.total_seconds()
fraud_df['user_tx_count'] = fraud_df.groupby('user_id')['user_id'].transform('count')

# --- Step 6: Encode categorical features ---
fraud_df = pd.get_dummies(fraud_df, columns=cat_cols, drop_first=True)

# --- Step 7: Scale numeric features safely ---
num_cols = ['purchase_value', 'age', 'time_since_signup', 'user_tx_count']
num_cols = [col for col in num_cols if col in fraud_df.columns]  # check exists
if len(num_cols) > 0 and len(fraud_df) > 0:
    scaler = StandardScaler()
    fraud_df[num_cols] = scaler.fit_transform(fraud_df[num_cols])

# --- Step 8: Prepare X, y ---
target_col = 'class'
if target_col in fraud_df.columns:
    X = fraud_df.drop(columns=[target_col])
    y = fraud_df[target_col]
else:
    raise ValueError(f"Target column '{target_col}' not found!")

print("Final X shape:", X.shape)
print("Final y shape:", y.shape)


Fraud DF shape: (0, 15)
IP DF shape: (138846, 3)
Empty DataFrame
Columns: [user_id, signup_time, purchase_time, purchase_value, device_id, age, ip_address, class, ip_int, lower_bound_ip_address, upper_bound_ip_address, time_since_signup, hour_of_day, day_of_week, user_tx_count]
Index: []
Final X shape: (0, 14)
Final y shape: (0,)


In [ ]:
 

# Step 1: Load the CSV properly
fraud_df = pd.read_csv('../data/raw/Fraud_Data.csv')
print("Fraud data loaded. Shape:", fraud_df.shape)
print(fraud_df.head())

# Step 2: Check for missing values in critical columns
print(fraud_df[['ip_address', 'purchase_value', 'class']].isna().sum())

# Step 3: Load IP mapping
ip_df = pd.read_csv('../data/raw/IpAddress_to_Country.csv')
print("IP mapping loaded. Shape:", ip_df.shape)
print(ip_df.head())

# Step 4: Merge carefully
# Convert IP to integer if needed, handle missing or malformed IPs
fraud_df['ip_address'] = fraud_df['ip_address'].fillna('0.0.0.0')
fraud_df['ip_int'] = fraud_df['ip_address'].apply(lambda x: sum([int(part)*(256**i) for i, part in enumerate(reversed(x.split('.')))]))
ip_df['lower_bound_ip_address'] = ip_df['lower_bound_ip_address'].astype(int)
ip_df['upper_bound_ip_address'] = ip_df['upper_bound_ip_address'].astype(int)

# Use a left join or interval merge (so you don’t drop rows)
# For simplicity, here we just map country if within range
def map_country(ip):
    mask = (ip_df['lower_bound_ip_address'] <= ip) & (ip <= ip_df['upper_bound_ip_address'])
    if mask.any():
        return ip_df.loc[mask, 'country'].values[0]
    else:
        return 'Unknown'

fraud_df['country'] = fraud_df['ip_int'].apply(map_country)
print("After mapping, sample data:\n", fraud_df[['ip_address','ip_int','country']].head())

# Step 5: Check if fraud_df is still empty
print("Shape after preprocessing:", fraud_df.shape)


Fraud data loaded. Shape: (151112, 11)
   user_id          signup_time        purchase_time  purchase_value  \
0    22058  2015-02-24 22:55:49  2015-04-18 02:47:11              34   
1   333320  2015-06-07 20:39:50  2015-06-08 01:38:54              16   
2     1359  2015-01-01 18:52:44  2015-01-01 18:52:45              15   
3   150084  2015-04-28 21:13:25  2015-05-04 13:54:50              44   
4   221365  2015-07-21 07:09:52  2015-09-09 18:40:53              39   

       device_id source browser sex  age    ip_address  class  
0  QVPSPJUOCKZAR    SEO  Chrome   M   39  7.327584e+08      0  
1  EOGFQPIZPYXFZ    Ads  Chrome   F   53  3.503114e+08      0  
2  YSSKYOSJHPPLJ    SEO   Opera   M   53  2.621474e+09      1  
3  ATGTXKYKUDUQN    SEO  Safari   M   41  3.840542e+09      0  
4  NAUITBZFJKHWW    Ads  Safari   M   45  4.155831e+08      0  
ip_address        0
purchase_value    0
class             0
dtype: int64
IP mapping loaded. Shape: (138846, 3)
   lower_bound_ip_address  upper_

AttributeError: 'float' object has no attribute 'split'

In [ ]:
if X.shape[0] > 0:
    smote = SMOTE(random_state=42)
    X_res, y_res = smote.fit_resample(X, y)
else:
    print("Cannot apply SMOTE: no data available.")


Cannot apply SMOTE: no data available.


In [ ]:
# Step 1: Check if fraud_df has data
if fraud_df.empty:
    raise ValueError("fraud_df is empty. Fix preprocessing before resampling.")

# Step 2: Make sure target column exists
if 'class' not in fraud_df.columns:
    raise ValueError("Target column 'class' not found in fraud_df.")

# Step 3: Prepare features and target
X = fraud_df.drop(columns=['class'])
y = fraud_df['class']

print("X shape:", X.shape)
print("y distribution:\n", y.value_counts())

# Step 4: Apply SMOTE only if we have data
if X.shape[0] > 0:
    smote = SMOTE(random_state=42)
    X_res, y_res = smote.fit_resample(X, y)
    print("After SMOTE, X_res shape:", X_res.shape)
    
    # Step 5: Save resampled data
    X_res.to_csv('../data/processed/fraud_features.csv', index=False)
    y_res.to_csv('../data/processed/fraud_target.csv', index=False)
    print("Task 1 complete: Processed data saved.")
else:
    print("No data available for SMOTE. Check preprocessing steps.")


ValueError: fraud_df is empty. Fix preprocessing before resampling.

In [ ]:
# Save processed data
X_res.to_csv('../data/processed/fraud_features.csv', index=False)
y_res.to_csv('../data/processed/fraud_target.csv', index=False)

print("Task 1 complete: Processed data saved.")



NameError: name 'X_res' is not defined